In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import func
from sqlalchemy import inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
# create engine to hawaii.sqlite
engine = create_engine(f"sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()

In [6]:
import os


In [7]:
# get the current working directory
cwd = os.getcwd()

# join the current working directory with the database filename
db_file = os.path.join(cwd,'hawaii.sqlite')

# check if the file exists
if os.path.exists(db_file):
    print(f"The file {db_file} exists.")
else:
    print(f"The file {db_file} does not exist.")

The file C:\Users\maclo\OneDrive\Desktop\Repositories\hw 10\Starter_Code\hawaii.sqlite exists.


In [8]:
file_path = "Resources/hawaii.sqlite"

if os.path.isfile(file_path):
    print("File exists")
else:
    print("File does not exist")

File exists


In [9]:
print(os.getcwd())


C:\Users\maclo\OneDrive\Desktop\Repositories\hw 10\Starter_Code


In [19]:
# reflect an existing database into a new model

# create an instance of automap_base
Base = automap_base()

# reflect the database schema
Base.prepare(engine, reflect=True)

In [20]:
# View all of the classes that automap found
Base.classes.keys()


['measurement', 'station']

In [25]:
from sqlalchemy import Column, Integer, String, Float, Date
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)


In [26]:
# Save references to each table
# Assign the measurement class to a variable called `Measurement`
Station = Base.classes.station

# Assign the Station class to a variable called `Station`
Station = Base.classes.station


AttributeError: type object 'Base' has no attribute 'classes'

In [ ]:
# Create our session (link) from Python to the DB
session= Session(engine)

# Exploratory Precipitation Analysis

In [27]:
most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(most_recent_date[0])

NameError: name 'session' is not defined

In [ ]:
# Find the most recent date in the data set.
RDate= session.query(func.max(measurment_table.date)).first()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
most_recent_date_obj = datetime.strptime(most_recent_date, '%Y-%m-%d').date()

# Calculate the date one year from the last date in data set.

one_year_ago_date = most_recent_date_obj - timedelta(days=365)
# Perform a query to retrieve the data and precipitation scores
results= session.query(Measurement.date,Measurement.prcp).filter(Measurement.date >= one_year_ago_date).all()
results

# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results, columns=['date','prcp'])
df.set_index('date', inplace=True)
df

# Sort the dataframe by date
df.sort_index(inplace=True)

# Use Pandas Plotting with Matplotlib to plot the data
# Plot the precipitation data
df.plot(figsize=(12,8))
plt.title('Last 12 Months of Precipitation Data')
plt.xlabel('Date')
plt.ylabel('Precipitation (inches)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
summary_stats = df.describe()
summary_stats

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
total_stations = session.query(Station).count()
total_stations

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
most_active = session.query(Measurement.station, func.count(Measurement.station)).\
              group_by(Measurement.station).\
              order_by(func.count(Measurement.station).desc()).all()

print(most_active)

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = 'USC00519281'

# Calculate the lowest, highest, and average temperature for the most active station
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
    filter(Measurement.station == most_active_station).all()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
last_twelve_months = session.query(Measurement.tobs).\
    filter(Measurement.station == most_active_station).\
    filter(Measurement.date >= (datetime.strptime('2016-08-18', '%Y-%m-%d') - timedelta(days=365))).\
    filter(Measurement.date <= datetime.strptime('2016-08-18', '%Y-%m-%d')).all()

In [ ]:
# Convert the results to a Pandas DataFrame and plot as a histogram
df = pd.DataFrame(last_twelve_months, columns=['tobs'])
df.plot.hist(bins=12, alpha=0.5)
plt.tight_layout()
plt.show()

# Close session

In [ ]:
# Close Session
session.close()